In [205]:
import pandas as pd
import pathlib

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

☐ 16  
☐ 17  
☐ 18  
☑ 19  

In [206]:
pathlib.Path('data').mkdir(parents=True, exist_ok=True)
if not pathlib.Path('data/yellow_tripdata_2022-01.parquet').exists():
    !curl -o data/yellow_tripdata_2022-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
if not pathlib.Path('data/yellow_tripdata_2022-02.parquet').exists():
    !curl -o data/yellow_tripdata_2022-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

In [207]:
df = pd.read_parquet('data/yellow_tripdata_2022-01.parquet')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN


In [208]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [209]:
print(f"There are {len(df.columns)} columns")

There are 19 columns


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

☐ 41.45  
☑ 46.45  
☐ 51.45  
☐ 56.45  

In [210]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].dt.total_seconds() / 60
duration_std = round(df['duration'].std(), 2)
print(f"Standard deviation of trip duration in January: {duration_std}")

Standard deviation of trip duration in January: 46.45


## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

☐ 90%  
☐ 92%  
☐ 95%  
☑ 98%  

In [211]:
nan_amount = df['duration'].isna().sum()
nan_amount

0

In [212]:
df_filtered = df[df['duration'].between(1, 60, inclusive='both')]
filtered_fraction = round(len(df_filtered) / len(df) * 100)
print(f"Percentage of records left after dropped outliers: {filtered_fraction}%")

Percentage of records left after dropped outliers: 98%


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

☐ 2  
☐ 155  
☐ 345  
☑ 515  
☐ 715  

In [213]:
X_cols = ['PULocationID', 'DOLocationID']
y_col = 'duration'

In [214]:
dict_vectorizer = DictVectorizer()
df_X = df_filtered[X_cols]
X_train = df_X.astype(str).to_dict(orient='records')
X_train = dict_vectorizer.fit_transform(X_train)
print(f"The dimensionality of this matrix is: {X_train.shape[1]}")

The dimensionality of this matrix is: 515


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

☑ 6.99  
☐ 11.99  
☐ 16.99  
☐ 21.99  

In [215]:
y_train = df_filtered[y_col]

model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
train_mrse = round(mean_squared_error(y_train, y_train_pred, squared=False), 2)
print(f"Train RMSE: {train_mrse}")

Train RMSE: 6.99


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2022). 

What's the RMSE on validation?

☑ 7.79  
☐ 12.79  
☐ 17.79  
☐ 22.79  

In [216]:
# Combine all the above into a function
def prepare_data(parquet_path, dict_vectorizer=None):
    df = pd.read_parquet(parquet_path)

    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].dt.total_seconds() / 60
    df_filtered = df[df['duration'].between(1, 60, inclusive='both')]

    X_cols = ['PULocationID', 'DOLocationID']
    y_col = 'duration'

    df_X = df_filtered[X_cols]
    X = df_X.astype(str).to_dict(orient='records')

    if not dict_vectorizer:
        dict_vectorizer = DictVectorizer()
        dict_vectorizer.fit(X)

    X = dict_vectorizer.transform(X)
    y = df_filtered[y_col]

    return X, y

X_test, y_test = prepare_data('data/yellow_tripdata_2022-02.parquet', dict_vectorizer)
y_test_pred = model.predict(X_test)
test_mrse = round(mean_squared_error(y_test, y_test_pred, squared=False), 2)
print(f"Test RMSE: {test_mrse}")

Test RMSE: 7.79
